# Data exploration 

---

Group name: O

---


## Introduction

*This section includes a short description of the data* 

## Setup

In [1]:
import pandas as pd
import altair as alt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## Data

## Import data

In [3]:
DATAGRADSTUDENTS = "https://raw.githubusercontent.com/FabioRaab/bigData-HW1/main/data/external/grad-students.csv"

In [11]:
PoliceKillingsData = https://raw.githubusercontent.com/FabioRaab/bigData-HW1/main/data/external/police_killings.csv"

SyntaxError: invalid syntax (4053392701.py, line 1)

### Data structure

In [4]:
df = pd.read_csv(DATAGRADSTUDENTS)
print(df)

df.info()

     Major_code                                       Major  \
0          5601                       CONSTRUCTION SERVICES   
1          6004           COMMERCIAL ART AND GRAPHIC DESIGN   
2          6211                      HOSPITALITY MANAGEMENT   
3          2201      COSMETOLOGY SERVICES AND CULINARY ARTS   
4          2001                  COMMUNICATION TECHNOLOGIES   
..          ...                                         ...   
168        5203                       COUNSELING PSYCHOLOGY   
169        5202                         CLINICAL PSYCHOLOGY   
170        6106     HEALTH AND MEDICAL PREPARATORY PROGRAMS   
171        2303                   SCHOOL STUDENT COUNSELING   
172        2301  EDUCATIONAL ADMINISTRATION AND SUPERVISION   

                          Major_category  Grad_total  Grad_sample_size  \
0    Industrial Arts & Consumer Services        9173               200   
1                                   Arts       53864               882   
2                    

### Data corrections

In [5]:
df['Grad_median'] = df['Grad_median'].astype("category")
df['Nongrad_median'] = df['Nongrad_median'].astype("category")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   Major_code                    173 non-null    int64   
 1   Major                         173 non-null    object  
 2   Major_category                173 non-null    object  
 3   Grad_total                    173 non-null    int64   
 4   Grad_sample_size              173 non-null    int64   
 5   Grad_employed                 173 non-null    int64   
 6   Grad_full_time_year_round     173 non-null    int64   
 7   Grad_unemployed               173 non-null    int64   
 8   Grad_unemployment_rate        173 non-null    float64 
 9   Grad_median                   173 non-null    category
 10  Grad_P25                      173 non-null    int64   
 11  Grad_P75                      173 non-null    float64 
 12  Nongrad_total                 173 non-null    int6

### Variable lists

In [7]:
# make a list of variables you want to use
var_list = ['Grad_median', 'Nongrad_median']

In [8]:
source = df[var_list]

In [9]:
source

,Grad_median,Nongrad_median
0,75000.0,65000.0
1,60000.0,48000.0
2,65000.0,50000.0
3,47000.0,41600.0
4,57000.0,52000.0
...,...,...
168,50000.0,40000.0
169,70000.0,46000.0
170,135000.0,51000.0
171,56000.0,42000.0


In [23]:
alt.Chart(source).mark_bar().encode(
    alt.X("Grad_median", bin=True),
    y='count()',
)

alt.Chart(...)

In [16]:
# contingency table for application type and homeownership.
pd.crosstab(df['Grad_median'],    # rows: Grad_median
            df['Nongrad_median'],    # columns: Nongrad_median
            margins=True)          # with total count

Nongrad_median,37000.0,38000.0,38700.0,40000.0,41600.0,42000.0,43000.0,43200.0,44000.0,45000.0,...,85000.0,87000.0,90000.0,92000.0,95000.0,98000.0,100000.0,107000.0,126000.0,All
Grad_median,,,,,,,,,,,,,,,,,,,,,
47000.0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
48000.0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
50000.0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
50100.0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
50750.0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110000.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3
111000.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
124000.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [14]:
chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('Grad_median',
        sort='-y' ),
    y=alt.Y('count(Nongrad_median)')
)
chart

alt.Chart(...)

In [13]:
#chart

#chart.properties( 
 #   title= 'Predicted Score in first halftime',
  #  width= 300,
   # height= 200)